# Dog Breeds Convolutional Classificator with TensorFlow

## Импорт библиотек

In [ ]:
from google.colab import drive
import shutil
import zipfile
import os
import pathlib
from PIL import Image
import tensorflow as tf
import numpy as np
from tensorflow import keras
from matplotlib import pyplot as plt

tf.test.gpu_device_name()

## Подключение GoogleDrive (Для Google Colab)

In [ ]:
drive.mount('/content/drive')

## Загрузка набора данных из GoogleDrive


In [ ]:
zip_file_path = '/content/drive/MyDrive/Datasets/StanfordDogs.zip'
shutil.copy(zip_file_path, '/content/')

unzip_dir = '/content/StanfordDogs/'
with zipfile.ZipFile('/content/StanfordDogs.zip', 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(os.listdir(unzip_dir))

## Валидация данных

In [ ]:
def check_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg'):
                filepath = os.path.join(root, file)
                try:
                    img_raw = tf.io.read_file(filepath)
                    img = tf.io.decode_image(img_raw, channels=3)
                    img_jpeg = tf.io.encode_jpeg(tf.cast(img, tf.uint8))
                except (tf.errors.InvalidArgumentError, ValueError) as e:
                    print(f'Error with image {filepath}: {e}')
                    os.remove(filepath)  # Удаление проблемного изображения
    print('All images are fine')


check_images(unzip_dir)

## Разбиение и загрузка данных

In [ ]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
    '/content/StanfordDogs/images/Images',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224),
    validation_split=0.2,
    subset='training',
    seed=1488
)

validation_dataset = keras.preprocessing.image_dataset_from_directory(
    '/content/StanfordDogs/images/Images',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224),
    validation_split=0.2,
    subset='validation',
    seed=1488
)

## Построение модели

Модель будет дообучаться на модели InceptionV3, обученной на основе набора данных ImageNet.

In [ ]:
inception_v3 = keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
print(f'InceptionV3 (No top) layers count: {len(inception_v3.layers)}')

In [ ]:
inception_v3.trainable = True

for layer in inception_v3.layers[:-200]:
    layer.trainable = False

data_augmentation = keras.Sequential([
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.2)
])

inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = inception_v3(x)
x = keras.layers.GlobalPooling2D()(x)
x = keras.layers.Dense(256)(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(120, activation='softmax')(x)

model = keras.Model(inputs, outputs)
model.summary()

## Компиляция модели

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## Создание обратных вызовов

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='/content/drive/MyDrive/Models/dogbreeds.keras',
        save_best_only=True
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
		patience=3
    )
]

## Обучение модели

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=50,
    callbacks=callbacks
)

## Визуализация обучения

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'go', label='Точность на этапе обучения')
plt.plot(epochs, val_accuracy, 'g', label='Точность на этапе проверки')
plt.title('Точность на этапах обучения и проверки')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'ro', label='Потери на этапе обучения')
plt.plot(epochs, val_loss, 'r', label='Потери на этапе проверки')
plt.title('Потери на этапах обучения и проверки')
plt.legend()
plt.show()

## Тестирование модели

In [ ]:
test_model = keras.models.load_model('/content/drive/MyDrive/Models/dogbreeds.keras')
test_loss, test_acc = test_model.evaluate(validation_dataset)
print(f'Test accuracy: {test_acc:.3f}')

## Просмотр формата предсказаний

In [ ]:
prediction = test_model.predict(validation_dataset.take(1))
print(f'Prediction format: {prediction}')